In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path

load_dotenv();


In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_qdrant import QdrantVectorStore
from langchain_core.output_parsers import StrOutputParser


In [5]:
pdf_path = Path(os.getcwd()).parent / "documents" / "doc_nodejs.pdf"
loader = PyPDFLoader(file_path=str(pdf_path))
docs = loader.load()


Ignoring wrong pointing object 268 0 (offset 0)
Ignoring wrong pointing object 309 0 (offset 0)


In [ ]:
print((docs[0]))


page_content='A PDF Reference for        The Complete Node.js Dev Course                Version 3.0' metadata={'producer': 'macOS Version 10.14.1 (Build 18B75) Quartz PDFContext', 'creator': 'Acrobat PDFMaker 17 for Word', 'creationdate': "D:20190227140340Z00'00'", 'author': 'Andrew Mead', 'moddate': "D:20190227140340Z00'00'", 'source': 'd:\\Python\\gen_ai\\documents\\doc_nodejs.pdf', 'total_pages': 125, 'page': 0, 'page_label': '1'}


In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
splitted_docs = splitter.split_documents(documents=docs)
print(len(splitted_docs))


333


In [ ]:
print(splitted_docs[0])


page_content='A PDF Reference for        The Complete Node.js Dev Course                Version 3.0' metadata={'producer': 'macOS Version 10.14.1 (Build 18B75) Quartz PDFContext', 'creator': 'Acrobat PDFMaker 17 for Word', 'creationdate': "D:20190227140340Z00'00'", 'author': 'Andrew Mead', 'moddate': "D:20190227140340Z00'00'", 'source': 'd:\\Python\\gen_ai\\documents\\doc_nodejs.pdf', 'total_pages': 125, 'page': 0, 'page_label': '1'}


In [ ]:
embedder = CohereEmbeddings(cohere_api_key=os.getenv("COHERE_API_KEY"), model="embed-english-v3.0")


In [ ]:
embedder.embed_query("Hey there")


In [ ]:
vector_store = QdrantVectorStore.from_documents(
    documents=[],
    embedding=embedder, 
    url="http://localhost:6333",
    collection_name="qdrant_langchain_rag"
)


In [ ]:
vector_store.add_documents(documents=splitted_docs)


In [ ]:
retriever = QdrantVectorStore.from_existing_collection(
    collection_name="qdrant_langchain_rag",
    url="http://localhost:6333",
    embedding=embedder
)


In [ ]:
llm = ChatGoogleGenerativeAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    model="gemini-1.5-flash"
)


In [ ]:
query = input(">>>>>>")


In [ ]:
result = retriever.similarity_search(query)
content = []
for doc in result:
    content.append(doc.page_content)

print(content)


In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant who respond to the query only from the given context.
     
     <context>
     {context}
     </context>
     
     """),
    ("human", "{query}")
])


In [ ]:
chain = prompt | llm | StrOutputParser()


In [ ]:
output = chain.invoke(input={
    "query": query,
    "context": content
})
print(output)


Based on the provided text, the `fs` module is a built-in Node.js module that provides functions for manipulating the file system.  The example shows its use with `writeFileSync` to write data to a file.
